In [9]:
import requests
from bs4 import BeautifulSoup
import random
import re
import heapq
from collections import Counter
import math
import time

Unfortunately, it was difficult for me to single out my favorite chapter, so I trusted a random selection.

In [12]:
url = 'https://www.hwlongfellow.org/poems_poem.php?pid=62'
response = requests.get(url)
soup_gen = BeautifulSoup(response.content, 'html.parser')
pids = []

for a in soup_gen.find_all('a', href=True):
    href = a['href']
    match = re.search(r'pid=(\d+)', href)
    if match:
        pid = match.group(1)
        pids.append(pid)
pids = list(set(pids))
if pids:
    random_pid = random.choice(pids)
    random_url = f"https://www.hwlongfellow.org/poems_poem.php?pid={random_pid}"

response_new = requests.get(random_url)
soup = BeautifulSoup(response_new.content, 'html.parser')

element = soup.find('span', class_='page-subtitle')
poem_text = None

for poem_id in ['poem', 'poem-content', 'content', 'main']:
    poem_text = soup.find('div', {'id': poem_id})
    if poem_text:
        break

if poem_text:
    full_text = poem_text.get_text()
    lines = [line.strip() for line in full_text.split('\n') if line.strip()]
    filtered_lines = []
    filtered_lines.append(element.get_text())
    for line in lines:
        if "The Song of Hiawatha 1855" in line:
            break  
        filtered_lines.append(line)
    cleaned_text = '\n'.join(filtered_lines)
    #print(cleaned_text)

#### Here is the code realisation of Haffman algorithm
Also there is the calculation of table size and other metrics

In [15]:
class HuffmanNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(text):
    # Building Haffman tree
    frequency = Counter(text)
    heap = [HuffmanNode(char, freq) for char, freq in frequency.items()]
    heapq.heapify(heap)
    
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = HuffmanNode(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        heapq.heappush(heap, merged)
    
    return heap[0]

def build_codes(node, current_code="", codes=None):
    # Building the code table
    if codes is None:
        codes = {}
    
    if node is None:
        return
    
    if node.char is not None:
        codes[node.char] = current_code
        return codes
    
    build_codes(node.left, current_code + "0", codes)
    build_codes(node.right, current_code + "1", codes)
    
    return codes

def huffman_compress(text):
    if not text:
        return "", {}
    
    root = build_huffman_tree(text)
    codes = build_codes(root)
    compressed_bits = ''.join(codes[char] for char in text)
    
    return compressed_bits, codes

def huffman_decompress(compressed_bits, codes):

    reverse_codes = {v: k for k, v in codes.items()}
    
    current_code = ""
    decompressed_text = ""
    
    for bit in compressed_bits:
        current_code += bit
        if current_code in reverse_codes:
            decompressed_text += reverse_codes[current_code]
            current_code = ""
    
    return decompressed_text

def calculate_table_size(codes):
    # Each character is stored as ASCII: 8 bits
    # Each code length: log2(max_code_length) bits
    if not codes:
        return 0

    max_code_length = max(len(code) for code in codes.values())
    bits_for_length = math.ceil(math.log2(max_code_length + 1))
    table_size_bits = 0
    
    for char, code in codes.items():
        table_size_bits += 8
        table_size_bits += bits_for_length
   
    return table_size_bits

start_time = time.perf_counter()
compressed_bits, codes = huffman_compress(cleaned_text)
all_time = time.perf_counter() - start_time
original_size = len(cleaned_text) * 8
compressed_size = len(compressed_bits)
table_size = calculate_table_size(codes)
total_compressed_size = compressed_size + table_size
ratio = original_size / compressed_size if compressed_size > 0 else 0
ratio_with_table = original_size / total_compressed_size if total_compressed_size > 0 else 0


#print("Code table:", codes)
print(f"Original size: {original_size} bites")
print(f"Compressed size: {compressed_size} bites" )
print(f"Table size: {table_size} bites" )
print(f"Compressed size with table size: {total_compressed_size} bites" )
print(f"Compression ratio: {ratio:.4f}")
print(f"Compression ratio with table: {ratio_with_table:.4f}")
print(f"Time of compressing: {all_time:.6f} seconds")

Original size: 32224 bites
Compressed size: 18762 bites
Table size: 732 bites
Compressed size with table size: 19494 bites
Compression ratio: 1.7175
Compression ratio with table: 1.6530
Time of compressing: 0.002561 seconds


Here is the process of decompressing, just for interest if two texts are matching

In [18]:
start_time = time.perf_counter()
decompressed_text = huffman_decompress(compressed_bits, codes)
all_time = time.perf_counter() - start_time
print(f"Text matching: {cleaned_text == decompressed_text}")
print(f"Time of decompressing: {all_time:.6f} seconds")

Text matching: True
Time of decompressing: 0.013420 seconds


#### Here is the code realisation of Lempel-Ziv-Welch algorithm
And a bit modified calculation of table size

In [21]:
def lzw_compress(text) :
    #Comressing via algorithm LZW
    used_chars = set(text)
    dictionary = {}
    next_code = 0

    for char in used_chars:
        dictionary[char] = next_code
        next_code += 1
    
    max_dict_size = 4096
    result_codes = []
    current_string = ""
    
    for char in text:
        new_string = current_string + char
        if new_string in dictionary:
            current_string = new_string
        else:
            result_codes.append(dictionary[current_string])
            if next_code < max_dict_size:
                dictionary[new_string] = next_code
                next_code += 1
            
            current_string = char

    if current_string:
        result_codes.append(dictionary[current_string])
    
    bit_sequence = codes_to_bits(result_codes, max_dict_size)
    
    return bit_sequence, dictionary

def codes_to_bits(codes, max_dict_size = 4096):

    if not codes:
        return ""
    
    max_code = max(codes) if codes else 0
    bits_per_code = math.ceil(math.log2(max_code + 1))

    max_bits = math.ceil(math.log2(max_dict_size))
    bits_per_code = min(bits_per_code, max_bits)

    bit_string = ""
    for code in codes:
        bin_code = bin(code)[2:]
        bin_code = bin_code.zfill(bits_per_code)
        bit_string += bin_code
    
    return bit_string

def lzw_decompress(bit_sequence, initial_dict):

    codes = bits_to_codes(bit_sequence, initial_dict)
    dictionary = {code: char for char, code in initial_dict.items()}
    dict_size = len(dictionary)
    max_dict_size = 4096
    
    result = []
    previous_code = codes[0]
    
    if previous_code not in dictionary:
        raise ValueError(f"Invalid first code: {previous_code}")
    
    result.append(dictionary[previous_code])
    
    for current_code in codes[1:]:
        if current_code in dictionary:
            current_string = dictionary[current_code]
        elif current_code == dict_size:
            current_string = dictionary[previous_code] + dictionary[previous_code][0]
        else:
            raise ValueError(f"Invalid code: {current_code}")
        
        result.append(current_string)
        
        if dict_size < max_dict_size:
            new_string = dictionary[previous_code] + current_string[0]
            dictionary[dict_size] = new_string
            dict_size += 1
        
        previous_code = current_code
    
    return ''.join(result)

def bits_to_codes(bit_sequence, initial_dict):
    if not bit_sequence:
        return []

    max_dict_size = len(initial_dict)
    max_initial_code = max(initial_dict.values()) if initial_dict else 255

    max_possible_code = max(max_initial_code, max_dict_size - 1)
    bits_per_code = math.ceil(math.log2(max_possible_code + 1))
    max_bits = math.ceil(math.log2(max_dict_size))
    bits_per_code = min(bits_per_code, max_bits)

    codes = []
    for i in range(0, len(bit_sequence), bits_per_code):
        code_bits = bit_sequence[i:i + bits_per_code]
        if len(code_bits) < bits_per_code:
            continue
        code = int(code_bits, 2)
        codes.append(code)
    
    return codes

def calculate_table_size_lwz(codes):
    if not codes:
        return 0
    max_string_length = max(len(string) for string in codes.keys())

    bits_for_length = math.ceil(math.log2(max_string_length + 1)) if max_string_length > 0 else 1
    
    table_size_bits = 0
    
    for string, code in codes.items():
        table_size_bits += 8 * len(string)
        table_size_bits += bits_for_length
        table_size_bits += 12
    
    return table_size_bits

In [23]:
start_time = time.perf_counter()
bit_sequence, result_codes = lzw_compress(cleaned_text)
all_time = time.perf_counter() - start_time
compressed_size_lwz = len(bit_sequence)
table_size_lwz = calculate_table_size_lwz(result_codes)
total_compressed_size_lwz = compressed_size_lwz + table_size_lwz
ratio_lwz = original_size / compressed_size_lwz if compressed_size_lwz > 0 else 0
ratio_with_table_lwz = original_size / total_compressed_size_lwz if total_compressed_size_lwz > 0 else 0

print(f"Original size: {original_size} bites")
print(f"Compressed size: {compressed_size_lwz} bites" )
print(f"Table size: {table_size_lwz} bites" )
print(f"Compressed size with table size: {total_compressed_size_lwz} bites" )
print(f"Compression ratio: {ratio_lwz:.4f}")
print(f"Compression ratio with table: {ratio_with_table_lwz:.4f}")
print(f"Time of compressing: {all_time:.6f} seconds")

Original size: 32224 bites
Compressed size: 19129 bites
Table size: 75384 bites
Compressed size with table size: 94513 bites
Compression ratio: 1.6846
Compression ratio with table: 0.3409
Time of compressing: 0.008879 seconds


Here is the process of decompressing, just for interest if two texts are matching
It was more difficult than with Haffman algorithm because the decompessing algorithm (another version without transfering the code table) gave unknown binary symbols from encoding.

In [26]:
start_time = time.perf_counter()
decompressed_text_lwz = lzw_decompress(bit_sequence, result_codes)
all_time = time.perf_counter() - start_time
print(f"Text matching: {cleaned_text == decompressed_text_lwz}")
print(f"Time of decompressing: {all_time:.6f} seconds")

Text matching: True
Time of decompressing: 0.006735 seconds


### Results
1. Huffman algorithm has better compression ratio (1.7175) than LWZ algorithm (1.6846), if we estimate without taking into account the size of the table.
2. Huffman algorithm's compression ratio (1.6530) is also better with the addition of the table (LWZ: 0.3409). This is because the Haffman table stores only pairs symbol - code, and the LWZ table stores much more combinations of symbols and its codes are alse demand more memory space.
3. The table size of Huffman algorithm is 732 bites.
4. The tible size of LWZ algorithm is 75384 bits, that is almost a hundred times more than Huffman table.
5. The processing time for Huffman algorithm compression is less than for LWZ one.
6. In general for this dataset Huffman algorithm is more effective.

### The algorithm for embedding the name of a state in some worldwide dataset for some ML task

##### Trigonometric encoding of geographical data of capitals.
__Step 1.__ Getting data about the country: its capital and latitude, longitude, height above sea level

__Step 2.__ Converting degrees to radians

__Step 3.__ Using sines and cosines to normalize latitude and longitude degrees to avoid a strong gap for actually close coordinates (e.g. +179 degrees and -179 degrees)

    For latitude (lat_sin, lat_cos), the sine changes from the South Pole to the North Pole, the cosine shows the distance from the poles
    
    For longitude (log_sin, log_cos), a pair of sine and cosine creates a cyclic representation
    
__Step 4.__ The height above sea level is standardized as a linear value

__Step 5.__ Creating the final feature vector: __[ lat_sin, lat_cos, log_sin, log_cos, standardized height above sea level ]__

The resulting vector for each capital describes its geographical location, taking into account the spherical closed shape of the Earth. This vector is the code of the corresponding capital.